In [ ]:
import subprocess
subprocess.check_call(["pip", "install", "selenium"])
subprocess.check_call(["pip", "install", "webdriver-manager"])
subprocess.check_call(["pip", "install", "requests"])

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd
import requests
import os

In [4]:
def abrir_navegador():
    """
    Abre o navegador e acessa o site do IBGE.

    :return: Navegador aberto.
    """
    opcoes = Options()
    opcoes.add_argument('--headless') # Rodar sem abrir o navegador
    # opcoes.add_argument('--auto-open-devtools-for-tabs') # Abrir o console do navegador
    # opcoes.add_argument('--start-maximized')
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    return navegador

def fechar_navegador(navegador):
    """
    Fecha o navegador.

    :param navegador: Navegador aberto.
    """
    navegador.quit()

def retornar_tabela(navegador):
    """
    Retorna a tabela com os dados de população por município do estado de São Paulo.

    :return: Tabela com os dados do site do IBGE.
    """
    elemento = (By.XPATH, '//*[@id="municipios"]/tbody[1]/tr/td[1]')
    WebDriverWait(navegador, 30).until(EC.presence_of_element_located(elemento))
    elemento = (By.XPATH, '//*[@id="municipios"]')
    tabela = WebDriverWait(navegador, 30).until(EC.presence_of_element_located(elemento))
    return tabela

def acessar_site(navegador, url):
    """
    Acessa o site pela URL.

    :param navegador: Navegador aberto.
    :param url: URL do site do IBGE.
    """
    navegador.get(url)
    tabela = retornar_tabela(navegador)
    return tabela


In [9]:
def dados_populacao(navegador):
    """
    Retorna um data frame com os dados de população por município do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site(navegador, 'https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=96385,96386,29765,29763,60036,47001')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df.drop(columns=['Gentílico'], inplace=True)
    df.drop(df.index[-1], inplace=True)
    df.rename(columns={'Municípios': 'cidade',
                       'População no último censo': 'populacao_censo',
                       'Pessoal ocupado': 'populacao_ocupado',
                       'População ocupada': 'porcentagem_populacao_ocupado',
                       'Salário médio mensal dos trabalhadores formais': 'salario_medio_mensal_trabalhadores_formais',
                       'PIB per capita': 'pib_per_capita',
                       'Densidade demográfica': 'densidade_demografica'}, inplace=True)
    df['cidade'] = df['cidade'].astype(str)
    df['populacao_censo'] = df['populacao_censo'].str.replace(' pessoas', '').astype(int)
    df['populacao_ocupado'] = df['populacao_ocupado'].str.replace(' pessoas', '').astype(int)
    df['porcentagem_populacao_ocupado'] = df['porcentagem_populacao_ocupado'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['salario_medio_mensal_trabalhadores_formais'] = df['salario_medio_mensal_trabalhadores_formais'].str.replace(' salários mínimos', '').str.replace(',', '.').astype(float)
    df['pib_per_capita'] = df['pib_per_capita'].str.replace(' R$', '').str.replace(',', '.').astype(float)
    df['densidade_demografica'] = df['densidade_demografica'].str.replace(' habitante por quilômetro quadrado', '').str.replace(',', '.').astype(float)
    return df

def dados_cidade(navegador):
    """
    Retorna um data frame com os dados referentes a municípios do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site(navegador, 'https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=30255,95335,60029,60031,29167')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df.drop(columns=['Gentílico'], inplace=True)
    df.drop(df.index[-1], inplace=True)
    df.rename(columns={'Municípios': 'cidade',
                       'Área da unidade territorial': 'area_territorio',
                       'Área urbanizada': 'area_urbanizada',
                       'Urbanização de vias públicas': 'urbanizacao_vias_publicas',
                       'Arborização de vias públicas': 'arborizacao_vias_publicas',
                       'Índice de Desenvolvimento Humano Municipal (IDHM)': 'idhm'}, inplace=True)
    df['cidade'] = df['cidade'].astype(str)
    df['area_territorio'] = df['area_territorio'].str.replace(' km²', '').str.replace(',', '.').astype(float)
    df['area_urbanizada'] = df['area_urbanizada'].str.replace(' km²', '').str.replace(',', '.').astype(float)
    df['urbanizacao_vias_publicas'] = df['urbanizacao_vias_publicas'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['arborizacao_vias_publicas'] = df['arborizacao_vias_publicas'].str.replace(' %', '').str.replace(',', '.').astype(float)
    df['idhm'] = df['idhm'].str.replace(',', '.').astype(float)
    return df

def dados_regiao():
    """
    Retorna um data frame com os dados referentes a regiões do estado de São Paulo.

    :return: Data frame com os dados de microrregiões, mesorregiões, regiões imediatas e regiões intermediárias do estado de São Paulo.
    """
    try:
        response = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados/SP/municipios/')

        if response.status_code == 200:
            tabela = response.json()

            df = pd.json_normalize(tabela)
            df = df[['nome', 'microrregiao.nome', 'microrregiao.mesorregiao.nome', 'regiao-imediata.nome', 'regiao-imediata.regiao-intermediaria.nome']]
            df.rename(columns={
                'nome': 'cidade',
                'microrregiao.nome': 'microrregiao',
                'microrregiao.mesorregiao.nome': 'mesorregiao',
                'regiao-imediata.nome': 'regiao_imediata',
                'regiao-imediata.regiao-intermediaria.nome': 'regiao_intermediaria'}, inplace=True)
            df['cidade'] = df['cidade'].astype(str)
            df['microrregiao'] = df['microrregiao'].astype(str)
            df['mesorregiao'] = df['mesorregiao'].astype(str)
            df['regiao_imediata'] = df['regiao_imediata'].astype(str)
            df['regiao_intermediaria'] = df['regiao_intermediaria'].astype(str)
            return df
        else:
            print(f"A solicitação falhou com status {response.status_code}")
    except Exception as e:
        print(f"Ocorreu um erro durante a solicitação: {e}")
    return None

def dados_cidade_abreviada():
    """
    Adiciona o nome abreviado das cidades para tratamento posterior no banco de dados.

    :return: Data frame com a nova coluna com o nome abreviado das cidades.
    """
    abreviadas = {
    'CAMPOS NOVOS PAUL.': 'Campos Novos Paulista',
    'ESPIRITO S.DO TURVO': 'Espírito Santo do Turvo',
    'ESPIRITO STO. PINHAL': 'Espírito Santo do Pinhal',
    'S.ADELIA': 'Santa Adélia',
    'S.ALBERTINA': 'Santa Albertina',
    'S.ANASTACIO': 'Santo Anastácio',
    'S.ANDRE': 'Santo André',
    'S.ANTONIO DA ALEGRIA': 'Santo Antônio da Alegria',
    'S.ANTONIO DE ARACANGUA': 'Santo Antônio do Aracanguá',
    'S.ANTONIO DE POSSE': 'Santo Antônio de Posse',
    'S.ANTONIO DO JARDIM': 'Santo Antônio do Jardim',
    'S.ANTONIO DO PINHAL': 'Santo Antônio do Pinhal',
    'S.BARBARA D OESTE': "Santa Bárbara d'Oeste",
    'S.BENTO DO SAPUCAI': 'São Bento do Sapucaí',
    'S.BERNARDO DO CAMPO': 'São Bernardo do Campo',
    'S.BRANCA': 'Santa Branca',
    'S.CAETANO DO SUL': 'São Caetano do Sul',
    'S.CARLOS': 'São Carlos',
    'S.CLARA D OESTE': "Santa Clara d'Oeste",
    'S.CRUZ DA CONCEICAO': 'Santa Cruz da Conceição',
    'S.CRUZ DA ESPERANCA': 'Santa Cruz da Esperança',
    'S.CRUZ DAS PALMEIRAS': 'Santa Cruz das Palmeiras',
    'S.CRUZ DO RIO PARDO': 'Santa Cruz do Rio Pardo',
    'S.ERNESTINA': 'Santa Ernestina',
    'S.EXPEDITO': 'Santo Expedito',
    'S.FE DO SUL': 'Santa Fé do Sul',
    'S.FRANCISCO': 'São Francisco',
    'S.GERTRUDES': 'Santa Gertrudes',
    'S.ISABEL': 'Santa Isabel',
    'S.JOAO DA BOA VISTA': 'São João da Boa Vista',
    'S.JOAO DE IRACEMA': 'São João de Iracema',
    'S.JOAO DO PAU D ALHO': "São João do Pau d'Alho",
    'S.JOAO DUAS PONTES': 'São João das Duas Pontes',
    'S.JOAQUIM DA BARRA': 'São Joaquim da Barra',
    'S.JOSE DA BELA VISTA': 'São José da Bela Vista',
    'S.JOSE DO BARREIRO': 'São José do Barreiro',
    'S.JOSE DO RIO PARDO': 'São José do Rio Pardo',
    'S.JOSE DO RIO PRETO': 'São José do Rio Preto',
    'S.JOSE DOS CAMPOS': 'São José dos Campos',
    'S.LOURENCO DA SERRA': 'São Lourenço da Serra',
    'S.LUCIA': 'Santa Lúcia',
    'S.LUIZ DO PARAITINGA': 'São Luiz do Paraitinga',
    'S.MANUEL': 'São Manuel',
    'S.MARIA DA SERRA': 'Santa Maria da Serra',
    'S.MERCEDES': 'Santa Mercedes',
    'S.MIGUEL ARCANJO': 'São Miguel Arcanjo',
    'S.PAULO': 'São Paulo',
    'S.PEDRO DO TURVO': 'São Pedro do Turvo',
    'S.PEDRO': 'São Pedro',
    'S.RITA D OESTE': "Santa Rita d'Oeste",
    'S.RITA PASSA QUATRO': 'Santa Rita do Passa Quatro',
    'S.ROQUE': 'São Roque',
    'S.ROSA DE VITERBO': 'Santa Rosa de Viterbo',
    'S.SALETE': 'Santa Salete',
    'S.SEBASTIAO DA GRAMA': 'São Sebastião da Grama',
    'S.SEBASTIAO': 'São Sebastião',
    'S.SIMAO': 'São Simão',
    'S.VICENTE': 'São Vicente'
    }
    df = pd.DataFrame(list(abreviadas.items()), columns=['cidade_abreviada', 'cidade'])
    df['cidade_abreviada'] = df['cidade_abreviada'].astype(str)
    df['cidade'] = df['cidade'].astype(str)
    return df

def dados_cidade_preenchida_errada():
    """
    Retorna um data frame com os dados referentes a cidades do estado de São Paulo que foram preenchidos errados.

    :return: Data frame com os dados de cidades do estado de São Paulo que foram preenchidos errados.
    """
    erradas = {
        'APARECIDA D OESTE': "Aparecida d'Oeste",
        'BIRITIBA-MIRIM': 'Biritiba Mirim',
        'BRODÓSQUI': 'Brodowski',
        'CAMPINA MONTE ALEGRE': 'Campina do Monte Alegre',
        'EMBU,EMBU GUAÇÚ': 'Embu-Guaçu',
        'ESTRELA D OESTE': "Estrela d'Oeste",
        'EUCLIDES DA CUNHA PTA': 'Euclides da Cunha Paulista',
        'FLORINIA': 'Florínea',
        'GUARANI D OESTE,GUARANI DOESTE': "Guarani d'Oeste",
        'IPAUCU,IPAUÇU': 'Ipaussu',
        'MIRANTE PARANAPANEMA': 'Mirante do Paranapanema',
        'MOGIGUACU,MOGI-GUACU,MOJI-GUACU': 'Mogi Guaçu',
        'MOGIMIRIM,MOGI-MIRIM': 'Mogi Mirim',
        'PALMEIRA D OESTE,PALMEIRA DOESTE': "Palmeira d'Oeste",
        'PIRAPORA BOM JESUS': 'Pirapora do Bom Jesus',
        'RIBEIRAO INDIOS': 'Ribeirão dos Índios',
        'SALMORAO': 'Salmourão',
        'SANTA BARBARA D OESTE,SANTA BÁRBARA D OESTE,SANTA BARBARA DOESTE,SANTA BÁRBARA DOESTE': "Santa Bárbara d'Oeste",
        'SANTA CLARA DOESTE': "Santa Clara d'Oeste",
        'SANTA ROSA DO VITERBO': 'Santa Rosa de Viterbo',
        'SANTO ANTONIO DA POSSE,SANTO ANTÔNIO DA POSSE': 'Santo Antônio de Posse',
        'SANTO ANTÔNIO DE ARACANGUÁ': 'Santo Antônio do Aracanguá',
        'SÃO JOÃO DO PAU D ALHO,SÃO JOÃO DO PAU DALHO': "São João do Pau d'Alho",
        'SAO LUIS DO PARAITINGA,SÃO LUÍS DO PARAITINGA': 'São Luiz do Paraitinga',
        'SEBASTIANOPOLIS SUL': 'Sebastianópolis do Sul',
        'SUD MENUCCI': 'Sud Mennucci',
        'TRES LAGOAS': 'Três Lagoas'
    }
    df = pd.DataFrame(list(erradas.items()), columns=['cidade_preenchida_errada', 'cidade'])
    df['cidade_preenchida_errada'] = df['cidade_preenchida_errada'].astype(str)
    df['cidade'] = df['cidade'].astype(str)
    return df

def coletar_dados():
    """
    Coleta os dados por município do estado de São Paulo.

    :return: Data frame com os dados por município do estado de São Paulo.
    """
    navegador = abrir_navegador()
    df_populacao = dados_populacao(navegador)
    df_cidade = dados_cidade(navegador)
    fechar_navegador(navegador)

    df_regiao = dados_regiao()
    df_cidade_abreviada = dados_cidade_abreviada()
    df_cidade_preenchida_errada = dados_cidade_preenchida_errada()

    df_unido = pd.merge(df_populacao, df_cidade, on='cidade')
    df_unido2 = pd.merge(df_unido, df_regiao, on='cidade')
    df_unido3 = pd.merge(df_unido2, df_cidade_abreviada, on='cidade', how="left")
    df = pd.merge(df_unido3, df_cidade_preenchida_errada, on='cidade', how="left")
    df = df[['cidade','cidade_abreviada', 'cidade_preenchida_errada', 'regiao_intermediaria', 'regiao_imediata', 'mesorregiao', 'microrregiao',
            'populacao_censo', 'populacao_ocupado', 'porcentagem_populacao_ocupado', 'salario_medio_mensal_trabalhadores_formais', 'pib_per_capita',
            'area_territorio', 'densidade_demografica', 'area_urbanizada', 'urbanizacao_vias_publicas', 'arborizacao_vias_publicas', 'idhm']]
    return df

def exportar_dados(dataframe):
    """
    Exporta os dados para um arquivo CSV.
    """
    nome_arquivo = 'dados_cidades.csv'
    diretorio = os.path.join(os.getcwd(), os.path.abspath('..\\'), 'Analises', 'Dados', nome_arquivo)
    dataframe.to_csv(diretorio, sep=';', encoding='latin1')


In [ ]:
df = coletar_dados()
exportar_dados(df)